In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Declaring Partial Derivatives
If you know additional information about the structure of partial derivatives in your component (for example, if an output does not depend on a particular input), you can use the `declare_partials`() method to inform the framework. This will allow the framework to be more efficient in terms of memory and computation (especially if using a sparse `AssembledJacobian`). This information should be declared in the setup_partials method of your component.

```{eval-rst}
    .. automethod:: openmdao.core.component.Component.declare_partials
        :noindex:
```

## Usage
Specifying that a variable does not depend on another. Note that this is not typically required, because by default OpenMDAO assumes that all variables are independent. However, in some cases it might be needed if a previous glob pattern matched a large set of variables and some sub-set of that needs to be marked as independent.

In [ ]:
def setup(self):
    self.add_input('x', shape=1)
    self.add_input('y1', shape=2)
    self.add_input('y2', shape=2)
    self.add_input('y3', shape=2)
    self.add_input('z', shape=(2, 2))

    self.add_output('f', shape=1)
    self.add_output('g', shape=(2, 2))

Declaring multiple derivatives using glob patterns (see https://docs.python.org/3.6/library/fnmatch.html).

In [ ]:
def setup(self):
    self.add_input('x', shape=1)
    self.add_input('y1', shape=2)
    self.add_input('y2', shape=2)
    self.add_input('y3', shape=2)
    self.add_input('z', shape=(2, 2))

    self.add_output('f', shape=1)
    self.add_output('g', shape=(2, 2))

Using the val argument to set a constant partial derivative. Note that this is intended for cases when the derivative value is constant, and hence the derivatives do not ever need to be recomputed in `compute_partials`. Here are several examples of how you can specify derivative values for differently-shaped partial derivative sub-Jacobians.

* Scalar (see $\frac{\partial f}{\partial x}$ )

* Dense Array (see $\frac{\partial g}{\partial y_1}$ )

* Nested List (see $\frac{\partial g}{\partial y_1}$ and $\frac{\partial g}{\partial y_3}$ )

* Sparse Matrix (see [Sparse Partial Derivatives doc](sparse_partials) for more details) (see $\frac{\partial g}{\partial y_2}$ and $\frac{\partial g}{\partial x}$)

In [ ]:
import numpy as np
import scipy as sp

import openmdao.api as om

class SimpleCompConst(om.ExplicitComponent):
    def setup(self):
        self.add_input('x', shape=1)
        self.add_input('y1', shape=2)
        self.add_input('y2', shape=2)
        self.add_input('y3', shape=2)
        self.add_input('z', shape=(2, 2))

        self.add_output('f', shape=1)
        self.add_output('g', shape=(2, 2))

    def setup_partials(self):
        # Declare derivatives

        self.declare_partials('f', ['y1', 'y2', 'y3'], dependent=False)
        self.declare_partials('g', 'z', dependent=False)

        self.declare_partials('f', 'x', val=1.)
        self.declare_partials('f', 'z', val=np.ones((1, 4)))
        # y[13] is a glob pattern for ['y1', 'y3']
        self.declare_partials('g', 'y[13]', val=[[1, 0], [1, 0], [0, 1], [0, 1]])
        self.declare_partials('g', 'y2', val=[1., 1., 1., 1.], cols=[0, 0, 1, 1], rows=[0, 2, 1, 3])
        self.declare_partials('g', 'x', val=sp.sparse.coo_matrix(((1., 1.), ((0, 3), (0, 0)))))

    def compute(self, inputs, outputs):
        outputs['f'] = np.sum(inputs['z']) + inputs['x']
        outputs['g'] = np.outer(inputs['y1'] + inputs['y3'], inputs['y2']) + inputs['x'] * np.eye(2)

    def compute_partials(self, inputs, partials):
        # note: all the partial derivatives are constant, so no calculations happen here.
        pass

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

model = om.Group(assembled_jac_type='dense')
problem = om.Problem(model=model)
problem.set_solver_print(0)

model.linear_solver = om.DirectSolver(assemble_jac=True)
model.add_subsystem('simple', SimpleCompConst(),
                    promotes=['x', 'y1', 'y2', 'y3', 'z', 'f', 'g'])
problem.setup()
problem.run_model()
totals = problem.compute_totals(['f', 'g'],
                                      ['x', 'y1', 'y2', 'y3', 'z'])

assert_near_equal(totals['f', 'x'], [[1.]])
assert_near_equal(totals['f', 'z'], np.ones((1, 4)))
assert_near_equal(totals['f', 'y1'], np.zeros((1, 2)))
assert_near_equal(totals['f', 'y2'], np.zeros((1, 2)))
assert_near_equal(totals['f', 'y3'], np.zeros((1, 2)))
assert_near_equal(totals['g', 'z'], np.zeros((4, 4)))
assert_near_equal(totals['g', 'y1'], [[1, 0], [1, 0], [0, 1], [0, 1]])
assert_near_equal(totals['g', 'y2'], [[1, 0], [0, 1], [1, 0], [0, 1]])
assert_near_equal(totals['g', 'y3'], [[1, 0], [1, 0], [0, 1], [0, 1]])
assert_near_equal(totals['g', 'x'], [[1], [0], [0], [1]])